In [41]:
import numpy as np
import rpy2
import rpy2.robjects as robjects
import pickle
from time import time

In [42]:
def store_results_dict(results, name):
    pickle.dump(results, open("./Results/{}.p".format(name), "wb" ))
    
def retrieve_results_dict(name):
    try:
        return pickle.load(open( "./Results/{}.p".format(name), "rb" ))
    except Exception as e:
        return None

### Data Generation

#### Raw Data

In [43]:
# GLOBAL CONFIG

# Var count
n_vars = 10

# Data types (default is standard normal)
binary_indeces = [1, 3, 6, 8, 9]
binarize = True

# Associations between vars an treat/outcome
treat_vars = [0,1,2,3,4,5,6,7]
outcome_vars = [0,1,2,3,4,8,9,10]

# Treat/outcome generation weights
assignment_weights = np.array([0, 0.8, -0.25, 0.6, -0.4, -0.8, -0.5, 0.7])
outcome_weights = np.array([-3.85, 0.3, -0.36, -0.73, -0.2, 0.71, -0.19, 0.26])
true_treat_effect = -0.4

def generate_data(n_samples=1000):
    # Generate 10 Random Vars
    # 1-4 are confounders: associated with outcome + treatment
    # 5-7 are exposure predictors
    # 8-10 are outcome predictors
    X = np.random.normal(loc=0.0, scale=1.0, size=(n_samples, n_vars))

    # Binarize specified vars if requested.
    if binarize:
        for var in binary_indeces:
            X[:, var-1] = (X[:, var -1] > 0).astype(int)

    # Add dummy for bias param     
    X = np.hstack([np.ones((n_samples, 1)), X])
    return X

In [44]:
# DEBUG
# X = generate_data(2000)
# X.shape

#### Assignment

In [45]:
# Create the models

assignment_models={}

def nonlinear_transform(X, B, quad_indeces):
    for quad_index in quad_indeces:
        quad = X[:, quad_index]**2
        X = np.hstack([X, quad.reshape(-1, 1)])
        B = np.append(B, B[quad_index])
    
    return X, B

def nonadditive_transform(X, B, interaction_indeces, interaction_weights=None):
    for interaction_index, var_indeces in enumerate(interaction_indeces):
        int_1, int_2 = var_indeces
        interaction_val = X[:, int_1]*X[:, int_2]
        
        if not interaction_weights:
            interaction_val = interaction_val*0.5
        else:
            interaction_val = interaction_val*interaction_weights[interaction_index]
            
        X = np.hstack([X, interaction_val.reshape(-1, 1)])
        B = np.append(B, B[int_1])
    
    return X, B

# Scenario 1
assignment_models["A_add_lin"] = lambda B, X: np.dot(X, B)

# Scenario 2:     
assignment_models["B_add_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2]))
# Scenario 3:
assignment_models["C_add_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2, 4, 7]))
# Scenario 4:
assignment_models["D_mild_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]))

# Scenario 5:
assignment_models["E_mild_nadd_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]), quad_indeces=[2]))
# Scenario 6
assignment_models["F_mod_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]))
# Scenario 7
assignment_models["G_mod_nadd_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]), 
                                                                            quad_indeces=[2, 4, 7]))

In [46]:
# Tests 
assert(set(assignment_models["A_add_lin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([9, 37]))

assert(set(assignment_models["B_add_mild_nlin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([33, 637]))

assert(set(assignment_models["C_add_mod_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([373, 13457]))

assert(set(assignment_models["D_mild_nadd_lin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([139.5, 3632]))

assert(set(assignment_models["E_mild_nadd_mild_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([163.5, 4232]))


#### Outcome and Assignment Functions

In [47]:
def get_assignments(B, X, n_samples, scenario="A_add_lin"):
    X_usable = X[:, treat_vars]
    
    # Calculate the probabilities of assignment
    linear_assignment_data = assignment_models[scenario](B, X_usable)
    p_treat = 1.0/(1+np.exp(-1*linear_assignment_data))

    # Assign
    rand = np.random.random(n_samples)
    assignments = (rand < p_treat).astype(int)
    
    return assignments

In [48]:
def get_outcomes(B, X, assignments, effect=true_treat_effect):
    X_usable = X[:, outcome_vars]
    return effect*assignments + np.dot(X_usable, B)

In [49]:
# DEBUG
# assignments = get_assignments(assignment_weights, X, "mild_nonaddititive_mild_nonlinear")
# outcomes = get_outcomes(outcome_weights, X, assignments)

### Analysis

In [50]:
from rpy2.robjects import IntVector, FloatVector, Formula
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

stats = importr('stats')
matching = importr('Matching')
snow = importr('snow')

Some code is going to >48 hours to ran. Lucky it's highly parellalisable so we can use a compute cluster. The one option is local to split across CPU cores. The better option is to go remote and explote 32 cores on multiple AWS machines.

On AWS, this is straightforward. Manually you need to port forward!. This allows the remote machine to connect to ports on the master via it's localhost loopback. 

```
# ~/.bash_profile
# Allow remote host to connect to local machine
# usage: $ remote_pfwd hostname {6000..6009}
function remote_pfwd {
  for i in ${@:2}
  do
    ssh -N -R $i:localhost:$i $1 &
  done
}
```
`remote_pfwd ubuntu@52.90.20.45 {11305..11307}`

In [51]:
AWS_MASTER_DNS="ip-172-31-42-147.ec2.internal"
AWS_SLAVE_1 = "ubuntu@ip-172-31-43-193.ec2.internal"
AWS_SLAVE_2 = "ubuntu@ip-172-31-81-244.ec2.internal"
AWS_MASTER_PORT_RANGE = list(range(11305, 11340))

class ClusterProvider(object):
    def __init__(self, n_nodes=8, remote_hosts=None, ports=None):
        if remote_hosts is None:
            self.cl = snow.makeSOCKcluster(["localhost"]*n_nodes)
        else:
            # Set the acceptable ports for connection
            # from the slaves
            if not ports:
                ports = AWS_MASTER_PORT_RANGE
            
            # Construct the connection string
            addresses = []
            for remote_host, n_nodes in remote_hosts:
                addresses+=[remote_host]*n_nodes
                
            self.cl = snow.makeSOCKcluster(addresses, rscript="Rscript", manual=False, snowlib="/usr/local/lib/R/site-library",
                                           port=IntVector(ports), master=AWS_MASTER_DNS, outfile="/dev/stdout", timeout=10)
    
    def get_cluster(self):
        return self.cl
    
    def kill_cluster(self):
        snow.stopCluster(self.cl)

In [52]:
# Local cluster
cluster_provider = ClusterProvider(n_nodes=8)

In [13]:
# Remote cluster
# cluster_provider = ClusterProvider(remote_hosts=[(AWS_SLAVE_1, 8)],
#                                     ports = list(range(11305, 11314)))

In [14]:
# Run this with True to kill the cluster
kill = False
if kill:
    cluster_provider.kill_cluster()

#### Estimators

Define methods which can process outcomes, assignments and covariate data into a treatment effect estimate. 

1. Logistic Regression
2. GenMatch
3. VAE

In [53]:
def get_propensity_scores(assignments, covariate_data):
    # Setup
    y = IntVector(assignments)
    fmla = Formula('y ~ X')
    env = fmla.environment
    
    # Run propensiy regression
    env['X'] = covariate_data
    env['y'] = y
    fit = stats.glm(fmla, family="binomial")
    
    # DEBUG: fit.rx("coefficients")
    return fit.rx2("fitted.values")

In [54]:
# 1. Logisic Regression Propensity Matching
def logistic_prop_matching_est(outcomes, assignments, covariate_data):
    
    propensity_scores = get_propensity_scores(assignments, covariate_data)
    
    # Run matching
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=propensity_scores,
        replace=True)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [55]:
# 2. GenMatch Matching
def genmatch_est(outcomes, assignments, covariate_data):
    
    # Get the singleton cluster
    cl = cluster_provider.get_cluster()
    
    # Add prop scores to covar data
    propensity_scores = np.array(get_propensity_scores(assignments, covariate_data))
    matching_data = np.hstack([covariate_data, propensity_scores.reshape(-1, 1)])
    
    start = time()
    gen_out = matching.GenMatch(
        Tr=IntVector(assignments),
        X=matching_data,
        BalanceMatrix=covariate_data,
        print_level=0,
        cluster=cl)
    print("GenMatch Time: ", time() - start)
    
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=matching_data,
        replace=True,
        Weight_matrix=gen_out)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [56]:
# DEBUG
# est = logistic_prop_matching_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

In [57]:
# DEBUG
# est = genmatch_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

#### Monte Carlo Runner Code

In [58]:
def get_data(n_samples, assignment_model):
    X = generate_data(n_samples)
    assignments = get_assignments(assignment_weights, X,
                                  n_samples, assignment_model)

    outcomes = get_outcomes(outcome_weights, X, assignments)
    
    return assignments, outcomes, X

def get_estimate(outcomes, assignments, covar_data, method, **args):
    return method(outcomes, assignments, covar_data, **args)

In [59]:
def run_simulation(runs=1000, n_samples=1000,
                   assignment_model="additive_linear",
                   estimator=logistic_prop_matching_est,
                   verbose=True,
                   **args):
    
    progress_tick = max(1, int(runs/10))
    results = np.zeros(runs)

    for i in range(runs):
        assignments, outcomes, covar_data = get_data(n_samples, assignment_model)
        
        covar_data = covar_data[:, 1:] #exclude bias term
        results[i] = get_estimate(outcomes, assignments,
                                  covar_data, estimator, **args)
        
        if i%progress_tick == progress_tick-1 and verbose:
            print("Done {} of {}".format(i+1, runs))
    
    bias = np.abs(np.mean((true_treat_effect-results)/true_treat_effect)*100)
    rmse = np.mean((true_treat_effect-results)**2)**0.5
    
    if verbose:
        print("\nRMSE", rmse)
        print("Bias", bias)
        print("===============\n\n")
    
    return {"RMSE": rmse, "Bias": bias}

In [60]:
# DEBUG:
# run_simulation(runs=20, n_samples=1000, assignment_model="D_mild_nadd_lin",
#               estimator=genmatch_est, verbose=True)

### Run MC trial

In [61]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

def get_store_name(models_being_run, est, runs, n_samples):
    # Storage
    
    if set(models_being_run) == set(assignment_model_names):
        store_name = "est_{}_runs_{}_n_{}".format(
            est.__name__,
            runs,
            n_samples)
    else:
        store_name = "est_{}_runs_{}_n_{}_models_{}".format(
            est.__name__,
            runs,
            n_samples,
            "_".join(models_being_run))
    
    return store_name

def run_test_battery(est,
                     store_name=None, 
                     runs=1000,
                     n_samples=1000,
                     models=assignment_models,
                     overwrite=False, verbosity=1,
                     **args):
    # Logging
    def printer(level, *args):
        if level <= verbosity:
            print(*args)
    
    # Storage config
    if store_name is None:
        store_name = get_store_name(models, est, runs, n_samples)
            
    results = retrieve_results_dict(store_name)

    if overwrite or (not results):
        printer(1, "No valid, existant results found. Beggining battery.\n")
        results = {}
        for model in models:
            printer(1, "Running: ", model)
            results[model] = run_simulation(
                                runs=runs,
                                n_samples=n_samples,
                                assignment_model=model,
                                estimator=est,
                                verbose=(verbosity==2),
                                **args)
            store_results_dict(results[model], store_name+"_checkpoint_"+model)
            printer(1, "Done.\n")

        store_results_dict(results, store_name)
    else:
        printer(1, "Displaying cached results.\n")
    
    printer(1, "Results")
    for model, results in results.items():
        printer(1, "Model: ", model)
        print(1, "Bias: ", results["Bias"])
        print(1, "RMSE: ", results["RMSE"], "\n")

### Run the Logistic Regression Battery

This one is easy. So we run on one machine.

In [24]:
run_test_battery(
    est=logistic_prop_matching_est,
    runs=1000,
    n_samples=1000)

Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.045874914703647685
1 RMSE:  0.07310500057973227 

Model:  B_add_mild_nlin
1 Bias:  3.1844355433209786
1 RMSE:  0.06588422028138122 

Model:  C_add_mod_nlin
1 Bias:  10.094350684204597
1 RMSE:  0.07650839711310455 

Model:  D_mild_nadd_lin
1 Bias:  6.720731771408928
1 RMSE:  0.08531717119502563 

Model:  E_mild_nadd_mild_nlin
1 Bias:  10.36168716658826
1 RMSE:  0.09094245826533698 

Model:  F_mod_nadd_lin
1 Bias:  3.1228082403965436
1 RMSE:  0.07605107262377982 

Model:  G_mod_nadd_mod_nlin
1 Bias:  11.830178367664905
1 RMSE:  0.07798212919046259 



### Run the GenMatch Battery

We split this across three machines using remote clusters.

In [25]:
gm_est = genmatch_est
gm_runs = 1000
gm_n_samples = 1000
gm_models_sets = [assignment_model_names[:3], assignment_model_names[3:5], assignment_model_names[5:]]
gm_files_to_be_produced = []

for model_set in gm_models_sets:
    gm_files_to_be_produced.append(get_store_name(model_set, gm_est, gm_runs, gm_n_samples))

gm_files_to_be_produced

['est_genmatch_est_runs_1000_n_1000_models_A_add_lin_B_add_mild_nlin_C_add_mod_nlin',
 'est_genmatch_est_runs_1000_n_1000_models_D_mild_nadd_lin_E_mild_nadd_mild_nlin',
 'est_genmatch_est_runs_1000_n_1000_models_F_mod_nadd_lin_G_mod_nadd_mod_nlin']

In [26]:
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[0],
    verbosity=2)

Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  5.5585826624331105
1 RMSE:  0.041571354846349606 

Model:  B_add_mild_nlin
1 Bias:  4.309919000663494
1 RMSE:  0.03799524129548324 

Model:  C_add_mod_nlin
1 Bias:  3.715796982487495
1 RMSE:  0.043206587873791204 



In [27]:
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[1],
    verbosity=2)

Displaying cached results.

Results
Model:  D_mild_nadd_lin
1 Bias:  2.30672600038697
1 RMSE:  0.040751955269481575 

Model:  E_mild_nadd_mild_nlin
1 Bias:  1.6356097465092616
1 RMSE:  0.0388547493767899 



In [28]:
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[2],
    verbosity=2)

Displaying cached results.

Results
Model:  F_mod_nadd_lin
1 Bias:  5.058677376450292
1 RMSE:  0.04404046330729526 

Model:  G_mod_nadd_mod_nlin
1 Bias:  3.185538423779952
1 RMSE:  0.04439998296725508 



In [62]:
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=[assignment_model_names[5]],
    verbosity=2)

No valid, existant results found. Beggining battery.

Running:  F_mod_nadd_lin


/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: rgenoud

  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: parallel

  warnings.warn(x, RRuntimeWarning)


GenMatch Time:  15.681255102157593
GenMatch Time:  26.463404893875122
GenMatch Time:  21.65285301208496
GenMatch Time:  21.174092054367065
GenMatch Time:  20.29900026321411
GenMatch Time:  17.52906608581543
GenMatch Time:  28.173863887786865
GenMatch Time:  19.248331785202026
GenMatch Time:  14.622676372528076
GenMatch Time:  12.128491163253784
GenMatch Time:  25.602908849716187
GenMatch Time:  8.594946146011353
GenMatch Time:  25.74100685119629
GenMatch Time:  19.381319046020508
GenMatch Time:  16.383818864822388
GenMatch Time:  19.406360864639282
GenMatch Time:  31.34773087501526
GenMatch Time:  17.743762969970703
GenMatch Time:  22.54181671142578
GenMatch Time:  14.646939039230347
GenMatch Time:  22.913729190826416
GenMatch Time:  18.108060121536255
GenMatch Time:  21.28918194770813
GenMatch Time:  15.41673493385315
GenMatch Time:  21.584607124328613
GenMatch Time:  18.667088985443115
GenMatch Time:  11.552114963531494
GenMatch Time:  23.564262866973877
GenMatch Time:  27.8159501552

GenMatch Time:  68.02664589881897
GenMatch Time:  59.431716203689575
GenMatch Time:  62.04587697982788
GenMatch Time:  47.07051110267639
GenMatch Time:  25.423840045928955
GenMatch Time:  98.06481194496155
GenMatch Time:  52.77937602996826
GenMatch Time:  35.392438888549805
GenMatch Time:  82.9883599281311
GenMatch Time:  40.36119985580444
GenMatch Time:  122.46567296981812
GenMatch Time:  53.76084089279175
GenMatch Time:  99.02014684677124
GenMatch Time:  60.91445279121399
GenMatch Time:  100.40463900566101
GenMatch Time:  80.20834708213806
GenMatch Time:  150.81942510604858
GenMatch Time:  49.707130908966064
GenMatch Time:  84.25667405128479
GenMatch Time:  78.40817284584045
GenMatch Time:  68.18096995353699
GenMatch Time:  70.66093897819519
GenMatch Time:  20.10014533996582
GenMatch Time:  57.358858823776245
GenMatch Time:  68.29918909072876
GenMatch Time:  106.43393683433533
GenMatch Time:  68.41304397583008
GenMatch Time:  50.83010983467102
GenMatch Time:  90.72648286819458
GenMat

GenMatch Time:  22.236494779586792
GenMatch Time:  14.12772512435913
GenMatch Time:  30.65456485748291
GenMatch Time:  49.82537078857422
GenMatch Time:  18.575150966644287
GenMatch Time:  20.440031051635742
GenMatch Time:  9.988296031951904
GenMatch Time:  24.97825312614441
GenMatch Time:  19.832139015197754
GenMatch Time:  23.200741052627563
GenMatch Time:  24.286547899246216
GenMatch Time:  15.574449062347412
GenMatch Time:  12.216270208358765
GenMatch Time:  6.947143316268921
GenMatch Time:  18.50043797492981
GenMatch Time:  22.799727201461792
GenMatch Time:  35.47003698348999
GenMatch Time:  23.558447122573853
GenMatch Time:  13.581045150756836
GenMatch Time:  19.21875786781311
GenMatch Time:  24.234949827194214
GenMatch Time:  30.125793933868408
GenMatch Time:  13.07127594947815
GenMatch Time:  18.595273971557617
GenMatch Time:  6.542855262756348
GenMatch Time:  20.700273990631104
Done 500 of 1000
GenMatch Time:  32.96023392677307
GenMatch Time:  10.435099124908447
GenMatch Time: 

GenMatch Time:  13.594770908355713
GenMatch Time:  17.054555892944336
GenMatch Time:  27.59350609779358
GenMatch Time:  25.436266899108887
GenMatch Time:  6.30153489112854
GenMatch Time:  17.3788800239563
GenMatch Time:  19.79133415222168
GenMatch Time:  13.482682943344116
GenMatch Time:  15.381102085113525
GenMatch Time:  17.119504928588867
GenMatch Time:  16.890983819961548
GenMatch Time:  10.41801404953003
GenMatch Time:  18.423481941223145
GenMatch Time:  23.191383123397827
GenMatch Time:  18.053980827331543
GenMatch Time:  29.52445411682129
GenMatch Time:  9.79004979133606
GenMatch Time:  6.167464256286621
GenMatch Time:  21.360733032226562
GenMatch Time:  26.078856229782104
GenMatch Time:  15.566630125045776
GenMatch Time:  6.233114004135132
GenMatch Time:  28.8541738986969
GenMatch Time:  16.80238699913025
GenMatch Time:  20.433099031448364
GenMatch Time:  10.524096012115479
GenMatch Time:  23.697975873947144
GenMatch Time:  21.512645959854126
GenMatch Time:  23.143213987350464


GenMatch Time:  13.550590991973877
GenMatch Time:  19.620457887649536
GenMatch Time:  23.477537155151367
GenMatch Time:  24.519282817840576
GenMatch Time:  14.105979204177856
GenMatch Time:  20.88282084465027
GenMatch Time:  19.90804409980774
GenMatch Time:  20.987627029418945
GenMatch Time:  16.226054906845093
GenMatch Time:  9.744450092315674
GenMatch Time:  11.678523778915405
GenMatch Time:  39.718364238739014
GenMatch Time:  14.748165130615234
GenMatch Time:  9.357563734054565
GenMatch Time:  34.135948181152344
GenMatch Time:  16.06557583808899
GenMatch Time:  18.104970932006836
GenMatch Time:  9.346842050552368
GenMatch Time:  9.144282102584839
GenMatch Time:  12.509736061096191
GenMatch Time:  27.030447959899902
GenMatch Time:  11.418156862258911
GenMatch Time:  15.867619037628174
GenMatch Time:  34.37984299659729
GenMatch Time:  18.0905020236969
GenMatch Time:  13.338121891021729
GenMatch Time:  17.506742000579834
GenMatch Time:  26.255047082901
GenMatch Time:  17.53310322761535

#### Combine results

In [34]:
gm_combined_name = get_store_name(assignment_model_names, gm_est, gm_runs, gm_n_samples)
linear_combined_name = get_store_name(assignment_model_names, logistic_prop_matching_est, 1000, 1000)

In [33]:
results = {}
for file in gm_files_to_be_produced:
    results.update(retrieve_results_dict(file))

store_results_dict(results, gm_combined_name)
results

{'A_add_lin': {'RMSE': 0.041571354846349606, 'Bias': 5.5585826624331105},
 'B_add_mild_nlin': {'RMSE': 0.03799524129548324, 'Bias': 4.309919000663494},
 'C_add_mod_nlin': {'RMSE': 0.043206587873791204, 'Bias': 3.715796982487495},
 'D_mild_nadd_lin': {'RMSE': 0.040751955269481575, 'Bias': 2.30672600038697},
 'E_mild_nadd_mild_nlin': {'RMSE': 0.0388547493767899,
  'Bias': 1.6356097465092616},
 'F_mod_nadd_lin': {'RMSE': 0.04404046330729526, 'Bias': 5.058677376450292},
 'G_mod_nadd_mod_nlin': {'RMSE': 0.04439998296725508,
  'Bias': 3.185538423779952}}

In [39]:
gm_results = retrieve_results_dict(gm_combined_name)
lin_results = retrieve_results_dict(linear_combined_name)

results = {
    "Linear": lin_results,
    "GenMatch": gm_results
}
for model in assignment_model_names:
    print(model, "\n")
    for matching in results.keys():
        print(matching)
        print("RMSE:", results[matching][model]["RMSE"], "Bias:", results[matching][model]["Bias"])
        
    print("==============")
    print()
    

A_add_lin 

Linear
RMSE: 0.07310500057973227 Bias: 0.045874914703647685
GenMatch
RMSE: 0.041571354846349606 Bias: 5.5585826624331105

B_add_mild_nlin 

Linear
RMSE: 0.06588422028138122 Bias: 3.1844355433209786
GenMatch
RMSE: 0.03799524129548324 Bias: 4.309919000663494

C_add_mod_nlin 

Linear
RMSE: 0.07650839711310455 Bias: 10.094350684204597
GenMatch
RMSE: 0.043206587873791204 Bias: 3.715796982487495

D_mild_nadd_lin 

Linear
RMSE: 0.08531717119502563 Bias: 6.720731771408928
GenMatch
RMSE: 0.040751955269481575 Bias: 2.30672600038697

E_mild_nadd_mild_nlin 

Linear
RMSE: 0.09094245826533698 Bias: 10.36168716658826
GenMatch
RMSE: 0.0388547493767899 Bias: 1.6356097465092616

F_mod_nadd_lin 

Linear
RMSE: 0.07605107262377982 Bias: 3.1228082403965436
GenMatch
RMSE: 0.04404046330729526 Bias: 5.058677376450292

G_mod_nadd_mod_nlin 

Linear
RMSE: 0.07798212919046259 Bias: 11.830178367664905
GenMatch
RMSE: 0.04439998296725508 Bias: 3.185538423779952

